In [1]:
%cd ..
%load_ext autoreload
%autoreload 2
# %load_ext nb_black
# notebook setup

from pathlib import Path
import sys

sys.path.insert(0, Path().absolute().parent.as_posix())

/Users/ruizhechao/Documents/NNforHJB


/Users/ruizhechao/Documents/NNforHJB/.venv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# load the data
import numpy as np

path = 'rawdata/raw_data/data/gauss_cos_31x31.npy'
data = np.load(path)
len(data)

961

In [3]:
# prepare the data
# data is a structured numpy array with fields: 'x', 'dv', 'v'
# convert to the dict format expected by model._prepare_data

data_dict = {
    "x": np.asarray(data["x"], dtype=np.float64),    # shape (N, 2)
    "v": np.asarray(data["v"], dtype=np.float64),    # shape (N,)
    "dv": np.asarray(data["dv"], dtype=np.float64),  # shape (N, 2)
}

## Test effect of different gamma with increased neurons added

In [4]:
gammas = [10.0, 1.0, 1e-1, 1e-2, 0]
alpha = 1e-5
power = 1.0
loss_weight_h1 = [1.0, 1.0]
loss_weight_l2 = [1.0, 0.0]

num_iterations = 10
num_insertions = 50
pruning_threshold = 1e-10

In [5]:
import torch
from scr.PDPA_v1 import PDPA_v1

pdpa_list_l2 = []
best_iteration_l2 = []
best_neurons_l2 = []
for gamma in gammas:
    pdpa = PDPA_v1(
        data=data_dict,
        alpha=alpha,
        gamma=gamma,
        power=power,
        activation=torch.relu,
        loss_weights=loss_weight_l2,
        verbose=True,
    )

    best_iteration, best_neurons = pdpa.retrain(
    num_iterations = num_iterations, 
    num_insertion= num_insertions, 
    threshold = pruning_threshold,
    decorrelation=True,
    verbose=True
    )

    pdpa_list_l2.append(pdpa)
    best_iteration_l2.append(best_iteration)
    best_neurons_l2.append(best_neurons)


2026-02-01 11:35:24.920 | INFO     | scr.model:__init__:72 - Model initialized
2026-02-01 11:35:24.920 | INFO     | scr.model:__init__:72 - Model initialized
2026-02-01 11:35:24.921 | INFO     | scr.model:_prepare_data:107 - Training set: 864 samples, Validation set: 97 samples
2026-02-01 11:35:24.925 | INFO     | scr.PDPA_v1:retrain:400 - Initialization
2026-02-01 11:35:26.526 | INFO     | scr.PDPA_v1:insertion:232 - insertion: 1st NOC profile - tried 50 candidates, accepted 47 (alpha=1e-05)
2026-02-01 11:35:26.530 | INFO     | scr.PDPA_v1:check_linearity_neurons:335 - Insertion: dropped=46 (zero=0, dependent=46), kept=1/47
2026-02-01 11:35:26.531 | INFO     | scr.PDPA_v1:retrain:410 - Iteration 1 - Starting Training...
2026-02-01 11:35:26.531 | INFO     | scr.model:_create_network:153 - Creating network with 1 neurons
2026-02-01 11:35:26.532 | INFO     | scr.model:train:265 - Starting network training session
2026-02-01 11:35:26.535 | INFO     | scr.model:train:338 - Epoch 0: Train L

In [6]:
best_neurons_l2

[7, 8, 7, 8, 1]

In [7]:
best_iteration_l2

[8, 9, 9, 8, 1]

In [8]:
# pdpa_list_h1 = []
# best_iteration_h1 = []

# for gamma in gammas:
#     pdpa = PDPA(
#         data=data_dict,
#         alpha=alpha,
#         gamma=gamma,
#         power=power,
#         activation=torch.relu,
#         loss_weights=loss_weight_h1,
#         verbose=False,
#     )

#     best_iteration = pdpa.retrain(
#     num_iterations = num_iterations, 
#     num_insertion= num_insertions, 
#     threshold = pruning_threshold,
#     verbose=False
#     )

#     pdpa_list_h1.append(pdpa)
#     best_iteration_h1.append(best_iteration)

In [9]:
import pickle

model_l2 = {
    "gammas": gammas,
    "pdpa_list_l2": pdpa_list_l2,
    "alpha": alpha,
    "power": power,
    "num_iteration": num_iterations,
    "num_insertion": num_insertions,
    "pruning_threshold": pruning_threshold,
    "best_iteration_l2": best_iteration_l2,
    "best_neurons_l2": best_neurons_l2
}

# model_h1 = {
#     "gammas": gammas,
#     "pdpa_list_h1": pdpa_list_h1,
#     "alpha": alpha,
#     "power": power,
#     "num_iteration": num_insertions,
#     "num_insertion": num_insertions,
#     "pruning_threshold": pruning_threshold,
#     "best_iteration_h1": best_iteration_h1
# }


with open("models/experiment_9_v1/pdpa_gausscos_model_l2_4.pkl", "wb") as f:
    pickle.dump(model_l2, f)

# with open("models/experiment/pdpa_gauscos_model_h1_0.pkl", "wb") as f:
#     pickle.dump(model_h1, f)